# <center>Class 6</center>

## Data Analysis using Pandas

In [ ]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

### Importing Match-level Data By Reading A CSV File

In [ ]:
file = os.path.join(os.pardir, 'data', 'E0.csv')

In [ ]:
df_matches = pd.read_csv(file)

<br>**What do we have?**   
Let's interpret the columns! 

In [ ]:
df_matches.head()

In [ ]:
df_matches.shape

Too many columns to list detailed info. 

In [ ]:
df_matches.info()

In [ ]:
df_matches[df_matches.HomeTeam == 'Arsenal'].head()

In [ ]:
df_matches[df_matches.HomeTeam == 'Aston Villa'].shape

In [ ]:
df_matches[df_matches.AwayTeam == 'Aston Villa'].shape

In [ ]:
for col in df_matches.columns:
    print(col)

**Limiting The Dataset**

In [ ]:
df_matches = df_matches[
    ['Date', 'HomeTeam', 'AwayTeam', 
     'FTHG', 'FTAG', 'HS', 'AS', 
     'B365H', 'B365A']
]

columns
- Date: date of the match
- HomeTeam: the team playing in its home stadium
- AwayTeam: the away team
- FTHG: full-time home goals
- FTAG: full-time away goals
- HS: goal attempts (shots) by the home team
- AS: goal attempts by the away team
- B365H: Bet365 odds for the home team before the match
- B365A: Bet365 odds for the away team before the match

In [ ]:
df_matches.head()

In [ ]:
df_matches.tail()

In [ ]:
df_matches.info()

In [ ]:
df_matches.Date.iloc[0]

In [ ]:
df_matches.Date.iloc[300]

<br>Date is a `string` which we may be better of with as a date-like object.

In [ ]:
pd.to_datetime(df_matches.Date.iloc[300]) # the default parser reads it as month/day/year by american standards if the parsing produces meaningful date

In [ ]:
pd.to_datetime(df_matches.Date.iloc[300], format = '%d/%m/%Y') # redefine date convention for British standards

In [ ]:
# How do we need to use the pd.to_datetime function ot make sure date strings are parsed properly? 

df_matches.Date = pd.to_datetime(df_matches.Date, format = '%d/%m/%Y')

In [ ]:
df_matches.info()

In [ ]:
# now it is - correctly - April
df_matches.Date.iloc[300]

<br>

### Match Stats Analysis - Plotting with Pandas `plot` method

In [ ]:
df_matches.describe()

In [ ]:
df_matches.describe().transpose()

Interesting fact: which team shot 6 goals on an away match?

In [ ]:
df_matches[df_matches.FTAG == 6]

<br>

**Soccer Is About Goals. Let's See The Goals!**

_Home goals_ can be directly read from the table.

<br>
<font color = '#646464'><b><i>How To Draw Charts From Pandas?</i></b></font>
<br>
<br>
There are two ways: 

- using the `plot()` method as described [here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html),
- calling the chart type, for instance `hist()` [directly](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.hist.html).

Pandas `plot()` methods offer a variety of customization options, the most of which can be found in the [matplotlib documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html). 

This is the basic chart output from Pandas `plot()` method.


In [ ]:
df_matches.FTHG.plot(
    kind = 'hist'
);

<br>We can add some customization to the plot for better aesthetics:
- figure size
- number of bins
- column alignment
- gap between columns
- chart title

In [ ]:
df_matches.FTHG.plot(
    kind = 'hist', bins = 7, 
    align = 'left', width = 0.95, figsize = (8,5),
    title = 'Home Goals Frequency',
);

_Away goals_ also

In [ ]:
df_matches.FTAG.plot(
    kind = 'hist', bins = [0,1,2,3,4,5,6,7], # define either number of bins (as an integer) or the bins themselves (as a list)
    align = 'left', width = 0.8, figsize = (8,5),
    title = 'Away Goals Frequency',
);

<br>Let's create _total goals_ as _TG_!

In [ ]:
# create 'TG' (total goals) column as a sum of goals by the home team and goals by the away team.
df_matches['TG'] = df_matches.FTHG + df_matches.FTAG

In [ ]:
df_matches.TG.plot(
    kind = 'hist', bins = list(range(0,11)), # list of bin categories
    align = 'left', width = 0.9, figsize = (8,5),
    title = 'Total Goals Frequency'
);

<br>Problem: x-axis value display is incomplete by default. We need to overwrite the default option using the `xticks = ` option, which accepts a `list` of ticks. 

In [ ]:
df_matches.TG.plot(
    kind = 'hist', bins = [x for x in range(0,11)], # list of bin categories with list comprehension
    align = 'left', width = 0.8, figsize = (8,5),
    title = 'Total Goals Frequency',
    xticks = [x for x in range(0,11)]
);

In [ ]:
df_matches.TG.value_counts()

In [ ]:
type(df_matches.TG.value_counts())

In [ ]:
df_matches.TG.value_counts().sort_index()

In [ ]:
df_matches[df_matches.TG == 9]

**Which teams are the top scorers?**

Answer:
- sum up goals
- grouped by team

Problem:
- there is _Home Team_ and _Away Team_

Solution:
- separate table for _home_ and _away_
- for potential later use we define both `mean` and `median`

<br>

Simple aggregation for one metric: mean. We are using `groupby()`. 

In [ ]:
df_matches.groupby('HomeTeam')['FTHG'].mean()

In [ ]:
pd.DataFrame(df_matches.groupby('HomeTeam')['FTHG'].mean())

Aggregation for multiple metrics: mean and median.

In [ ]:
df_goal_stats = df_matches.groupby('HomeTeam').agg(
    {'FTHG': [np.mean, np.median],
    'FTAG': [np.mean, np.median]}
)

How can we interpret this table?

In [ ]:
df_goal_stats

What does the value 0.894737 mean at the 'Arsenal' row?

What is this weird table structure? These are [*multilevel columns*](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html#hierarchical-indexing-multiindex). We won't cover the delicacies here, but it is a lot of fun &#128518;.

In [ ]:
df_goal_stats.columns

In [ ]:
for col in df_goal_stats.columns:
    print(col)

<br>**Which teams are the top scorers at home?**

In [ ]:
# Columns in a multi-level structure are selected as a tuple
df_goal_stats.sort_values(by = ('FTHG','mean'), ascending = False) # Default is 'ascending' so we have to add 'False' as input parameter to put the top scorers first.

<br>**Which teams got the least goals (has the best defense) at home?**

In [ ]:
df_goal_stats.sort_values(by = ('FTAG','mean'), ascending = True)

<br>**Which teams are the top scorers away?**

In [ ]:
df_goal_stats_away = df_matches.groupby('AwayTeam').agg(
    {'FTHG': [np.mean, np.median],
    'FTAG': [np.mean, np.median]}
)

In [ ]:
df_goal_stats_away.sort_values(by = ('FTAG','mean'), ascending = False)

<br>**Which teams aget the fewst goals from the home team while they are playing away?**

In [ ]:
df_goal_stats_away.sort_values(by = ('FTHG','mean'), ascending = True)

**Most agressive teams home and away**

We assume that agressive teams make many attempts to shoot, so _home shots_ (HO) and _away shots_ (AO) is a measure of ambitious tactic. 

In [ ]:
# Average shots at home per team.
df_shot_stats_home = df_matches.groupby('HomeTeam').agg(
    {'HS': [np.mean, np.median],
    'AS': [np.mean, np.median]}
)

In [ ]:
# Average shots away by team.
df_shot_stats_away = df_matches.groupby('AwayTeam').agg(
    {'HS': [np.mean, np.median],
    'AS': [np.mean, np.median]}
)

Most aggressive at home

In [ ]:
df_shot_stats_home.sort_values(by = ('HS','mean'), ascending = False)

Most agressive away. 

In [ ]:
df_shot_stats_away.sort_values(by = ('AS','mean'), ascending = False)

Let's have a look at Manchester City! 

In [ ]:
df_matches[df_matches.AwayTeam == 'Man City']

<br>***What is the typical home and away shot number per match? Do teams play more on the defensive when playing away?***

Home

In [ ]:
df_matches['HS'].plot(
    kind = 'hist',  bins = [x for x in range(0,32)], # bins from 0 to 31
    figsize = (8,5), width = 0.8, align = 'left',
    title = 'Home shot distribution over the year'
);

Same graph but with a slightly different syntax and look.

In [ ]:
df_matches['HS'].hist(
    bins = list(range(0,32)), # bins from 0 to 31
    figsize = (8,5), width = 0.9, align = 'left'
)
plt.title('Home shot distribution over the year');

Away

In [ ]:
df_matches['AS'].hist(
    bins = list(range(0,32)), # bins from 0 to 31
    figsize = (8,5), width = 0.9, align = 'left'
)
plt.title('Away shot distribution over the year');

Does this make sense? Do we expect temas playing at home - on average- being more agressive than those playing away? Think and argue why this can or cannot be the right pattern.

<br>Let's compare home shots vs away shots distributions directly!

In [ ]:
df_matches[['HS', 'AS']].plot(kind = 'hist',
    bins = [x for x in range(0,32)], # bins from 0 to 31
    figsize = (8,5), width = 0.9, align = 'left', 
    alpha = 0.75, color = ['royalblue', 'dimgray'],
    title = 'Home and away shot distribution over the year'
);

<br>Interesting fact: when plotting *home shots* on a team-by-team basis we can't see this unimodal, normal-like distribution. Why is that?

In [ ]:
df_matches[['HomeTeam', 'HS']].hist(
    by = 'HomeTeam', 
    figsize = (16,12), bins = 31, width = 0.8);

<details><summary><b>Click here for the answer</b></summary>
    
<br>This is, eventually, the **[Central Limit Theorem](https://en.wikipedia.org/wiki/Central_limit_theorem) in action!**

</details>

#### Formal Statistical Testing



Here we would like to formally test ***whether teams are equally agressive at home than away***. (We can test whether they are *more* agressive at home than away, but for now we are simply testing the equality.) 

We measure agressiveness by comparing the typical number of shots by the home teams against their _corresponding_ away teams. For this, we are using the so-called **[t_test](https://en.wikipedia.org/wiki/Student%27s_t-test)**. Since for each _home shot value_ there is a corresponding _away shot value_, we are using the [paired t-test](https://en.wikipedia.org/wiki/Student%27s_t-test#Dependent_t-test_for_paired_samples)

In statistical testing we are trying to find proof that our _null hypothesis_ ($H_0$) is ***not true***. For this, we split the event space into to subspaces, ($H_0$) and ($H_1$):   
<center>
    $H_0: \overline{X}_{homeshots} =  \overline{X}_{awayshots}$  <br> 
    $H_1: \overline{X}_{homeshots} \neq \overline{X}_{awayshots}$
</center><br>
 
 
This is a very important concept, atlhough at first blush it goesn against common sense. We formulate our assumption in a way that ***what we would like to reject goes into $H_0$, and what we would like to prove goes into $H_1$***. Rejecting $H_0$ means accepting $H_1$ (the alternative), while failing to reject $H_0$ simply means we have just not found sufficient evidence to do so. 

In case $H_0$ cannot be rejected, then the corresponding `t-test` produces a 'low' t-value. Here all we can say is that _we have **not found evidence** that the typical number of shots per game by the home team is higher than the away team_. This would mean that teams playing home do not prove more agressive than their counterparts. 

If we find evidence, that ($H_0$) is not true (so most likely the opposite $H_1$ is true), we will see a 'high' t-value (or _t-statistic_). In this case then we can say that _we have **found evidence** for home teams, **on average**, playing more agressively, than away teams_. 

The t-test formula is
<center>
  <font size = 4px>$t = \frac{\overline{X}_{homeshots} - \overline{X}_{awayshots}}{s_d  /\sqrt{n}}$</font>
</center>
<br>where $s_d$ is the standard deviation of the difference between home shots and away shots per match, and $n$ is the sample size (here 380 matches). 

OK, but what is _high_ t-value and what is _low_ t_value? This is what `p-value` is for. 

The _p-value_ is the likelihood of accepting $H_1$ when in fact $H_0$ is true. A <font color = 'blue'>low p_value indicates that the non-zero difference between the two population averages is not the result of some random extremity, but they are truly different</font>. If _p < 5%_, then $H_0$ is most likely false and can be rejected. 

For this, we use the `ttest_rel()` method of the [scipy package](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html#scipy.stats.ttest_rel). 

In [ ]:
from scipy import stats

In [ ]:
stats.ttest_rel(df_matches.HS, df_matches.AS)

For better output and presentation we can define a new variable as the result of the function, and call the `statistic` and the `pvalue` attributes of this new object. 

In [ ]:
t_stat = stats.ttest_rel(df_matches.HS, df_matches.AS)

In [ ]:
print('t-value: {:.3f}'.format(t_stat.statistic))

p-value as _float_

In [ ]:
print('p_value: {:.3f}'.format(t_stat.pvalue))

p-value as _percent_

In [ ]:
print('p_value: {:.1%}'.format(t_stat.pvalue))


<br>So are they really different? Do teams, _on average_, play more agressively at home or not? What do you think?

<br>**Some more shot stats**

Quantiles: deciles & quartiles.

Refresher: 
- '*deciles*' split the population into 10 equal-sized groups. 
    - The first decile (D1) is a number which is larger than 10 pct of the population and smaller than 90 percent of the population. This is *not the minimal* value but is a *relatively low value* within the population. 
    - D9 can be interpreted in the same fashion. 
- '*quartiles*' split the population into 4 equal-sized groups. 
    - The first quartile (Q1) is the value which is larger than 25 percent of the population. 
    - The *interquartile range* (the [Q1- Q3] interval is the mid 50 pct of the population. 
- The 5th decile (D5) or the second quartile (Q2) equals the *median* (Me) which splits the population into two equal-sized group. The median is one of the [*measures of central tendency*](https://www.abs.gov.au/websitedbs/D3310114.nsf/Home/Statistical+Language+-+measures+of+central+tendency).  

The `quantile()` method returns the value of a given quantile. For instance `quantile(0.1)` returns the *10th percentile*, or the *1st decile*. 

In [ ]:
df_matches['HS'].quantile(0.1)

In [ ]:
print('10 percent of matches had less than {} shots by the home team in the 2021-22 season.'.format(df_matches['HS'].quantile(0.1)))

<br>Preparing for the decile distribution table:

In [ ]:
for i in range(1,10):
    print('D' + str(i))

In [ ]:
for i in range(1,10):
    print(i, df_matches['HS'].quantile(i/10))

In [ ]:
for i in range(1,10):
    print(i, round(df_matches['HS'].quantile(i/10),1))

In [ ]:
data = []
for i in range(1,10):
    l = ['D' + str(i), round(df_matches.HS.quantile(i/10),1), round(df_matches.AS.quantile(i/10),1)]
    data.append(l)

In [ ]:
data

In [ ]:
df_deciles = pd.DataFrame(data = data, columns = ['decile', 'home shots', 'away shots'])

How can we interpret this table?

In [ ]:
df_deciles

What is the problem with the table above in terms of the visual represantation of the patterns in our data?

In [ ]:
df_deciles.sort_values(by = 'decile', ascending = False)

In [ ]:
df_deciles.query("decile == 'D5'") # which value is this?

A simple **bar chart**.

In [ ]:
df_deciles.plot(
    kind = 'bar',
    x = 'decile', y = ['home shots', 'away shots'],
    figsize = (8,5), color = ['grey', 'royalblue'], 
    yticks = [x*2 for x in range(1,11)],               # check out how the list of y-axis ticks gets generated
    title = 'Home and away shot distribution'
);

*Note*: you can spend hours decorating your charts. To do that, use the [matplotlib color code guide](https://matplotlib.org/stable/gallery/color/named_colors.html). 

**Boxplot**.

In [ ]:
df_matches[['HS', 'AS']].plot(
    kind = 'box'
);

In [ ]:
df_matches[['HS', 'AS']].plot(
    kind = 'box',
    figsize = (8,5), widths = [0.25, 0.25], 
    color = {'boxes': 'grey', 'medians': 'orange', 'whiskers': 'black', 'caps': 'black'}, patch_artist=True
);

For those of artistic talents: this is how [to decorate boxplots](https://matplotlib.org/stable/gallery/statistics/boxplot.html) and other plots. 

<br>***How good are the sport betting platforms in asessing match outcomes?***

In [ ]:
df_matches.head(10)

In [ ]:
df_matches[['B365H', 'B365A']].plot(
    kind = 'box'
);

In [ ]:
df_matches[['B365H', 'B365A']].describe().transpose()

In [ ]:
df_matches[df_matches.B365H == df_matches.B365H.max()]

In [ ]:
df_matches[df_matches.B365A == df_matches.B365A.max()]

Let's create three new variables:
- 'RESULT' as FTHG - FTAG
- 'SHOT_DIFF' as HS - AS
- 'ODDS_DIFF' as B365H - B365A

In [ ]:
df_matches['RESULT'] = df_matches.FTHG - df_matches.FTAG # if positive, home team wins
df_matches['SHOT_DIFF'] = df_matches.HS - df_matches.AS # if positive, home team is more offensive
df_matches['ODDS_DIFF'] = df_matches.B365H - df_matches.B365A # if NEGATIVE, home team win pays less (home team has higher odds)

In [ ]:
df_matches.head()

<br>Do betting sites get match outcomes right? Odds vs final result.

Note: See how $x$ and $y$ are used *both* as _variable names_ and _variable values_ in this short script! 

In [ ]:
x = 'ODDS_DIFF' 
y = 'RESULT'

df_matches.plot(
    kind = 'scatter', 
    x = x, y = y,
    figsize = (8,5), 
    title = (f'{y} vs {x} in the 2021-22 season') # check out the f-strings!
);

Why does this graph look so unfortunate? This is because of differences in [cardinality](https://www.merriam-webster.com/dictionary/cardinality). 

Same chart for _difference in odds_ **vs** _difference between home shots and away shots_.

In [ ]:
x = 'ODDS_DIFF' 
y = 'SHOT_DIFF'

df_matches.plot(
    kind = 'scatter', 
    x = x, y = y,
    figsize = (8,5), 
    title = (f'{y} vs {x} in the 2021-22 season')
);

<br>

### Enriching The Dataset With Financial Data

In [ ]:
df_financials = pd.read_csv('https://raw.githubusercontent.com/eddwebster/football_analytics/master/data/capology/raw/premier-league/2021-2022/all_premier-league_2021-2022_last_updated_05092021.csv', index_col = 0)

***A reality check of what we have***

In [ ]:
df_financials.head()

Rewriting column names where necessary.

In [ ]:
df_financials.rename(
    columns = {
        'Weekly GrossBase Salary(IN GBP)' : 'Weekly_Gross',
        'Annual GrossBase Salary(IN GBP)' : 'Annual_Gross',
        'Pos.': 'Pos',
        'EstimatedGross Total(IN GBP)' : 'Estimated_Gross'
    }, 
    inplace = True
)

In [ ]:
df_financials.head()

In [ ]:
df_financials.shape

In [ ]:
df_financials.Season.unique() # Really 2021-22 only 

In [ ]:
df_financials.groupby('Team')['Player'].count()

In [ ]:
df_financials[df_financials.Team == 'Aston Villa']

Do we have the same teams in the two tables?

In [ ]:
df_financials.Team.unique()

In [ ]:
df_matches.HomeTeam.unique()

Remember: for comparison on memberships, we can use Python `set` variable type. It is a **collection** which is unordered, unchangeable, and unindexed. The most important aspect of sets that the *can't have two items of the same value*.

In [ ]:
# What does the match stat table has which is not included in the financials table?
set(df_financials.Team) - set(df_matches.HomeTeam)

In [ ]:
# And the other way around?
set(df_matches.HomeTeam) - set(df_financials.Team)

Let's handle the issue by replacing the new versions with the old one by using the `replace()` method.

In [ ]:
dc_replacements = {
    'Manchester City': 'Man City', 
    'Manchester United': 'Man United', 
    'Wolverhampton': 'Wolves'
}

In [ ]:
df_financials['Team'].replace(dc_replacements, inplace = True)

In [ ]:
set(df_financials.Team) - set(df_matches.HomeTeam)

In [ ]:
set(df_matches.HomeTeam) - set(df_financials.Team)

In [ ]:
df_financials.head()

<br>Converting '*Weekly_Gross*' into a number!

<br>Converting *Weekly_Gross* to an integer.

In [ ]:
df_financials['Weekly_Gross'].iloc[0]

In [ ]:
df_financials['Weekly_Gross'].iloc[0].split(' ')

In [ ]:
df_financials['Weekly_Gross'].iloc[0].split(' ')[1]

In [ ]:
df_financials['Weekly_Gross'].iloc[0].split(' ')[1].replace(',','')

In [ ]:
int(df_financials['Weekly_Gross'].iloc[0].split(' ')[1].replace(',',''))

Now let's apply this step-by-step transformation to each element in the 'Length' column using `map` and `lambda functions`. 

In [ ]:
df_financials['Weekly_Gross'] = df_financials['Weekly_Gross'].map(lambda x: int(x.split(' ')[1].replace(',','')))

In [ ]:
df_financials.head()

In [ ]:
df_financials.info()

<br>
What is the order in terms of player remuneration?

In [ ]:
# weekly average
df_financials.groupby('Team')['Weekly_Gross'].mean().sort_values(ascending = False)

In [ ]:
# weekly sum; affected by player number in our data table!
df_financials.groupby('Team')['Weekly_Gross'].sum().sort_values(ascending = False)

<br>

***Combining The Datasets: Merging/Joining***

In [ ]:
df_matches.head()

In [ ]:
df_financials.head()

In [ ]:
df_team_values = pd.DataFrame(df_financials.groupby('Team')['Weekly_Gross'].mean())

In [ ]:
df_team_values

<br>

In SQL it would be:
```SQL
SELECT * 
FROM df_matches
LEFT JOIN df_team_values
ON df_matches.HomeTeam = df_team_values.index

```

<br>

In [ ]:
df = pd.merge(df_matches, df_team_values, how = 'left', left_on = 'HomeTeam', right_index = True)

In [ ]:
df.head()

In [ ]:
df.columns = [x for x in df.columns[:-1]] + ['HomeTeam_Weekly_Pay']

In [ ]:
df.head()

In [ ]:
df = pd.merge(df, df_team_values, how = 'left', left_on = 'AwayTeam', right_index = True)

In [ ]:
df.head()

In [ ]:
# df.columns = [x for x in df.columns[:-1]] + ['AwayTeam_Weekly_Pay']
df.rename({'Weekly_Gross': 'AwayTeam_Weekly_Pay'}, axis = 'columns', inplace = True)

In [ ]:
df.head()

In [ ]:
df['HomeTeam_Excess_Weekly_Pay'] = df.HomeTeam_Weekly_Pay - df.AwayTeam_Weekly_Pay

In [ ]:
df.head()

<br>

#### Question: how important are the differences in player emunerations for betting sites to determine odds?

Do teams with higher average player salaries have lower betting payout ratios?

In [ ]:
df.plot(
    kind = 'scatter', x = 'HomeTeam_Excess_Weekly_Pay', y = 'ODDS_DIFF',
    figsize = (8,5)
);

#### Plotting using `seaborn`

In [ ]:
import seaborn as sns

A visual representation of a pattern of a *linear relationship* between x (excess pay for the home team) and y (the net odds in favov of the home team). We will call it a `linear regression` in DA2.

In [ ]:
sns.regplot(
    data = df,
    x = 'HomeTeam_Excess_Weekly_Pay', y = 'ODDS_DIFF', 
    marker= '.',
    fit_reg= True);

With some additional customization.

In [ ]:
sns.regplot(
    data = df,
    x = 'HomeTeam_Excess_Weekly_Pay', y = 'ODDS_DIFF', 
    marker= '.',
    fit_reg= True, line_kws = {'color' : 'k'})
plt.axhline(y = 0, color = 'k');

Some inquiries into two extreme data points. 

In [ ]:
df[df.HomeTeam_Excess_Weekly_Pay == df.HomeTeam_Excess_Weekly_Pay.max()]

In [ ]:
df[df.ODDS_DIFF == df.ODDS_DIFF.max()]